The transition matrix is only 900x900, very easy to calculate, so we automatically have the transition matrix for any number of steps. Once we have the full probability matrix for the final state, the expectation of a cell being empty is simply the product of the probability of each flea not being there, i.e., one minus the probability at that cell.

In [1]:
# A brute force Monte-Carlo simulation useful for simulating small
# boards to validate and shake out bugs.

import random

import numpy as np


def monte_carlo(m, n, steps, repeat_count):
    cummulated = 0
    for _ in range(repeat_count):
        state = np.ones((m, n), dtype=int)
        for _ in range(steps):
            old_state = state
            state = np.zeros((m, n), dtype=int)
            for i in range(m):
                for j in range(n):
                    choices = [
                        (x, y)
                        for x, y in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
                        if 0 <= x < m and 0 <= y < n
                    ]
                    for _ in range(old_state[i][j]):
                        x, y = random.choice(choices)
                        state[x][y] += 1
        cummulated += m * n - np.count_nonzero(state)
    return cummulated / repeat_count


monte_carlo(4, 4, 10, 10_000)

5.6636

In [2]:
import numpy as np


def generate_transition_matrix(m, n):
    t = np.zeros((m * n, m * n))
    for i in range(m):
        for j in range(n):
            choices = [
                (x, y)
                for x, y in [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
                if 0 <= x < m and 0 <= y < n
            ]
            for x, y in choices:
                t[i * n + j][x * n + y] = 1 / len(choices)
    return t


def calculate_empty_expectation(m, n, steps):
    t = generate_transition_matrix(m, n)
    tt = np.linalg.matrix_power(t, steps)
    ss = np.ones((m * n, m * n)) - tt
    return sum(ss[:, j].prod() for j in range(m * n))


def main():
    print("{:.6f}".format(calculate_empty_expectation(30, 30, 50)))


if __name__ == "__main__":
    main()


330.721154
